In [3]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy
import pandas as pd
#from xgboost import XGBClassifier

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [5]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]

In [6]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams(aggregated_data)
# Run script

In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:-0.5443811951009836
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)

exported_pipeline.fit(np.array(X), np.array(Y))


Pipeline(memory=None,
     steps=[('stackingestimator-1', StackingEstimator(estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=2,
       min_child_weight=4, missing=None, n_estimators=100, nthread=1,
       objective='binary:logistic...ty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [9]:
# perform simluation of 2017 NCAA tourney
print("Simulating 2017 NCAA tourney")
submission_data = []
tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
tourney_2017_teams = tourney_seeds[tourney_seeds['Season'] == 2017]['Team']

stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']
print("Predicting matchups.")
tourney_2017_teams.sort_values()

for team_1 in tourney_2017_teams:
    for team_2 in tourney_2017_teams:
        if team_1 < team_2:
            prediction = mm_predictor.predict_winner(
                team_1, team_2, exported_pipeline, 2017, stat_fields)
            label = '2017_' + str(team_1) + '_' + str(team_2)
            submission_data.append([label, prediction[0][0]])

Simulating 2017 NCAA tourney
Predicting matchups.


In [23]:
teamNames = pd.read_csv('./ncaa-data/Teams.csv')

team_id_map = {}
for index, row in teamNames.iterrows():
    team_id_map[row['TeamID']] = row['TeamName']

teamTeamWinChanceMap = {}
    
for line in submission_data:
    title = line[0].split('_')
    team1 = team_id_map[int(title[1])]
    team2 = team_id_map[int(title[2])]
    winRate = str(round(line[1]* 100,2))
    if team1 not in teamTeamWinChanceMap:
        teamTeamWinChanceMap[team1] = {}
    teamTeamWinChanceMap[team1][team2] = float(winRate)
    #print(team1 + ' has a ' + str(winRate) + ' percent chance of beating ' + team2)

print(teamTeamWinChanceMap)

{'Villanova': {'Virginia': 48.32, 'Wisconsin': 46.97, 'Virginia Tech': 49.67, 'West Virginia': 44.47, 'Xavier': 59.54, 'Wichita St': 34.37, 'Wake Forest': 56.25, 'Winthrop': 46.52}, 'Duke': {'Villanova': 21.74, 'Florida': 37.06, 'Virginia': 40.65, 'SMU': 22.56, 'South Carolina': 49.06, 'Wisconsin': 38.4, 'Virginia Tech': 41.81, 'Marquette': 48.13, 'Providence': 44.28, 'USC': 44.43, 'UNC Wilmington': 33.53, 'ETSU': 37.55, 'New Mexico St': 38.54, 'Troy': 48.46, "Mt St Mary's": 58.58, 'New Orleans': 56.96, 'Gonzaga': 21.62, 'Florida St': 36.24, 'West Virginia': 36.12, 'Notre Dame': 35.26, 'Maryland': 41.81, "St Mary's CA": 30.02, 'Northwestern': 44.28, 'Vanderbilt': 45.05, 'VA Commonwealth': 34.98, 'Xavier': 50.89, 'Princeton': 36.12, 'FL Gulf Coast': 41.51, 'North Dakota': 48.7, 'S Dakota St': 57.29, 'Kansas': 26.33, 'Louisville': 34.85, 'Oregon': 27.53, 'Purdue': 34.74, 'Iowa St': 35.81, 'Michigan': 35.12, 'Miami FL': 42.23, 'Michigan St': 51.06, 'Oklahoma St': 46.32, 'Rhode Island': 35

In [15]:
print("Writing %d results." % len(submission_data))
with open('./ncaa-data/submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'pred'])
    writer.writerows(submission_data)

Writing 2278 results.
